<h1>
    Imports

In [4]:
import numpy as np
import random

<h1>
    Loading Data
   

In [15]:
X_train = np.genfromtxt('X_train-2.csv', delimiter = ",")
y_train = np.genfromtxt('y_train-1.csv')
X_test = np.genfromtxt('X_test.csv', delimiter = ",")

lambda_input = int(random.randrange(2**15)%10)
sigma2_input = float(random.randrange(2**15)%10 + 1)


<h1>
Part 1

PART 1: In this part you will implement the $\ell_2$-regularized least squares linear regression algorithm we have been discussing (ridge regression). Recall from the lectures that this takes the form:

$$w_{RR} = \arg\min_w \|y - Xw\|^2 + \lambda\|w\|^2.$$

Your task will be to write code that takes in data $y$ and $X$ and outputs $w_{RR}$ for an arbitrary value of $\lambda$.

In [40]:
## Solution for Part 1
def part1():
    ## Input : Arguments to the function
    ## Return : wRR, Final list of values to write in the file
    
    lam = lambda_input
    X = X_train
    y = y_train.reshape(-1, 1)
    
    n, d = X.shape
    I = np.eye(d)
    wRR = np.linalg.inv(X.T @ X + lam * I) @ X.T @ y
    
    wRR_list = wRR.flatten().tolist()
    
    return wRR_list
    
    pass

In [42]:
w_list = part1()
print(w_list)

[-0.48942113544740873, -0.14326120023253536, -0.6750692667582878, -4.441432987230459, 0.05761700738240542, 2.649350829516372, -0.001491690381728028]


<h1>
    Part 2

In the same code, you will also implement the active learning procedure discussed in Lecture 5. For this problem, we will provide you with an arbitrary setting of $\lambda$ and $\sigma^2$ and ask you to provide us with the first 10 locations you would measure from a set $\mathcal{D} = \{x\}$ given a set of measured pairs (y, X). Please look over the slides carefully to remind yourself about the sequential evolution of the sets $\mathcal{D}$ and (y,X).

In [43]:
def part2():
    ## Input: Uses global X_train, y_train, X_test, lambda_input, sigma2_input
    ## Return: List of indices (0-based) of the 10 selected points from X_test
    
    # Convert everything to 2D/column format
    X = X_train
    y = y_train.reshape(-1, 1)
    X_test_pool = X_test.copy()

    lam = lambda_input
    sigma2 = sigma2_input

    selected_indices = []
    available_indices = list(range(X_test.shape[0]))

    for _ in range(10):
        XTX_inv = np.linalg.inv(X.T @ X + lam * np.eye(X.shape[1]))
        
        # Compute variance for each x in X_test_pool
        variances = []
        for i in range(X_test_pool.shape[0]):
            x = X_test_pool[i].reshape(1, -1)
            var = sigma2 * (x @ XTX_inv @ x.T)
            variances.append(var.item())  # convert 1x1 matrix to scalar

        # Find index of test point with max variance
        max_idx = np.argmax(variances)

        # Record the index of selected point (original index from X_test)
        selected_indices.append(available_indices[max_idx])

        # Add selected point to training set
        X = np.vstack([X, X_test_pool[max_idx]])
        y = np.vstack([y, [[0]]])  # We don't know y, but it's required structurally

        # Remove the selected point from test pool
        X_test_pool = np.delete(X_test_pool, max_idx, axis=0)
        del available_indices[max_idx]

    return selected_indices


In [44]:
selected = part2()
print(selected)


[24, 15, 23, 26, 31, 30, 4, 19, 33, 14]
